In [ ]:
#모델을 짤거임.

In [110]:
from keras.models import Input, Model
from keras.layers import Dropout, Activation, Add,Bidirectional,LSTM,MaxPooling1D, Conv1D,BatchNormalization, Flatten, Dense,Embedding

In [2]:
from gensim.models.word2vec import Word2Vec
import glob
from konlpy.tag import Twitter 
import re
import pandas as pd

C:\Users\ie-45\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
train_data // 
test_data //

In [ ]:
'''
Input 
embedding_layer
conv2d()
lstm()
lstm()
lstm()
dense()
dense()
Model
'''

In [160]:
word_model = Word2Vec.load("./model/last_train_model.model")

In [77]:
pretrained_weights = word_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape
max_sequence_length = 2000 #train 최대 길이에 맞추고 test는 그길이보다 길면 잘라줌


C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [55]:
word_model.wv.syn0.shape

C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(26252, 200)

In [72]:
vocab_size

26252

In [104]:
#첫모델 아직 input 안됨
def model_creation(): 
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(vocab_size,embedding_size,
        weights=[word_model.wv.syn0],
        input_length=max_sequence_length,trainable=False)(input_layer) 
    
    #embedding_layer_2 = np.expand_dims(X, axis=2)
    c0 = Dropout(0.2)(embedding_layer)
    c1 = (Conv1D(64,kernel_size = 5, border_mode='same'))(c0)
    c2 = (BatchNormalization())(c1)
    c3 = (Activation('relu'))(c2)
    c4 = (MaxPooling1D(pool_size=(4)))(c3)
    c5 = (Dropout(0.25))(c4)

    c6 = (Conv1D(64,3, border_mode='same'))(c5)
    c7 = (BatchNormalization())(c6)
    c8 = (Activation('relu'))(c7)
    c9 = (MaxPooling1D(pool_size=(4)))(c8)
    c10 = (Dropout(0.25))(c9)

    left1 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', input_shape=(32, 32)))(c10)
    left2 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', input_shape=(32, 32)))(left1)

    right1 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(32, 32), go_backwards=True))(c10)
    right2 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(32, 32), go_backwards=True))(right1)
    
    
    merged = Add()([left2,right2])

    h1 = (Flatten())(merged)
    h2 = (Dense(512,activation='relu'))(h1)
    h3 = (Dense(256,activation='relu'))(h2)
    h4 = (Dense(128,activation='relu'))(h3)

    h5 = (Dense(2,activation='sigmoid'))(h4)


    model = Model(inputs = input_layer , outputs = output)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [112]:
def model_creation(): 
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(vocab_size,embedding_size,
        weights=[word_model.wv.syn0],
        input_length=max_sequence_length,trainable=False)(input_layer) 
    
    
    left1 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', input_shape=(32, 32)))(embedding_layer)
    left2 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', input_shape=(32, 32)))(left1)

    right1 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(32, 32), go_backwards=True))(embedding_layer)
    right2 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(32, 32), go_backwards=True))(right1)
    #embedding_layer_2 = np.expand_dims(X, axis=2)
    merged = Add()([left2,right2])
    
    c0 = Dropout(0.2)(merged)
    c1 = (Conv1D(64,kernel_size = 5, border_mode='same'))(c0)
    c2 = (BatchNormalization())(c1)
    c3 = (Activation('relu'))(c2)
    c4 = (MaxPooling1D(pool_size=(4)))(c3)
    c5 = (Dropout(0.25))(c4)

    c6 = (Conv1D(64,3, border_mode='same'))(c5)
    c7 = (BatchNormalization())(c6)
    c8 = (Activation('relu'))(c7)
    c9 = (MaxPooling1D(pool_size=(4)))(c8)
    c10 = (Dropout(0.25))(c9)

    h1 = (Flatten())(c10)
    h2 = (Dense(512,activation='relu'))(h1)
    h3 = (Dense(256,activation='relu'))(h2)
    h4 = (Dense(128,activation='relu'))(h3)

    output = (Dense(2,activation='sigmoid'))(h4)


    model = Model(inputs = input_layer , outputs = output)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [114]:
model = model_creation()

C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.
C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, activation="tanh", input_shape=(32, 32), unit_forget_bias=True, units=256, kernel_initializer="uniform", recurrent_initializer="uniform", recurrent_activation="sigmoid")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, activation="tanh", input_shape=(32, 32), unit_forget_bias=True, units=256, kernel_initializer="uniform", recurrent_initializer="uniform", recurrent_activation="sigmoid")`
  del sys.path[0]
C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_la

In [169]:
def tagging(one_row):
    hangle = re.compile('[^ ㄱ-ㅣ가-힣0-9]+')
    result = hangle.sub('',one_row)
    result = tagger.pos(result,norm=True,stem = True)
    return result

def data_preprocessing(one_row):
    data = tagging(one_row)
    r = []
    results = []
    jung_bok = ""
    for (word, pumsa) in data:
        if not pumsa in ["Josa", "Eomi", "Punctuation",'Number'] and jung_bok !=word:
            r.append(word)
            jung_bok = word
    results.append((" ".join(r)).strip())  
    return results[0]

def data_making(input_data):
    data = pd.read_csv(input_data,encoding='euc-kr').iloc[:,1:]
    data['times'] =data['times'].str[:10]
    data['times'] = data['times'].apply(lambda x : x.replace('.',"-"))
    data['str'] = data['titles'].str[:]+data['articles']
    del data['titles']
    del data['articles']
    data['str'] = data['str'].apply(data_preprocessing)
    return data
def dataframe_preprocessing(data,i,d):
    dataframe = data_making(data)
    data = dataframe['str'][i].split()
    data = word_to_vec(data)
    return data,dataframe[d][i]
    
def word_to_vec(data):
    word_vec_list = []
    for txt in data:
        try:
            vec = word_model.wv[txt]
            word_vec_list.append(vec)
        except:
            pass
    return word_vec_list

In [170]:
train_data_dir = glob.glob("./train_data/*.csv")

In [171]:
train_data_dir[0]

'./train_data\\2017.06.01_2017.12.31_hmscience.csv'

In [172]:
#train_data
train_one_row,train_one_label = dataframe_preprocessing(train_data_dir[0],0,'d1')

In [173]:
train_one_label

0

In [129]:
train_data_1_d1_label = train_data_1['d1']

In [135]:
train_data_1 = data_preprocessing(train_data_1.iloc[0,5])